# Day 7: No Space Left On Device

[*Advent of Code 2022 day 7*](https://adventofcode.com/2022/day/7) and [*solution megathread*](https://www.reddit.com/zesk40)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2022/07/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2022%2F07%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')


%load_ext nb_mypy
%nb_mypy On

Version 1.0.4


In [2]:
import common


downloaded = common.refresh()
%store downloaded >downloaded

%load_ext pycodestyle_magic
%pycodestyle_on

Writing 'downloaded' (dict) to file 'downloaded'.


## Part One

In [3]:
from IPython.display import HTML

HTML(downloaded['part1'])

## Comments

...

In [4]:
from IPython.display import display

testdata = """$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k""".splitlines()

inputdata = downloaded['input'].splitlines()

In [59]:
from typing import Dict


class Node(object):
    def __init__(self, name: str, size: int = 0):
        self.name = name
        self.size = size

    def __repr__(self):
        return f'{self.size} {self.name}'


class DirNode(Node):
    def __init__(self, name: str):
        self.contents: Dict[str, Node] = dict()
        super().__init__(name)

    def __repr__(self):
        return '\n'.join(
            [f'{self.size} {self.name}:'] +
            [subnode.__repr__()
             for subnode in self.contents.values()
             if subnode.name != '..'])

In [48]:
root = DirNode('')
root.contents['foobar'] = Node('foobar', 30)
root.contents['barbar'] = DirNode('barbar')

In [51]:
barbar = root.contents['barbar']
if isinstance(barbar, DirNode):
    barbar.contents['carcar'] = Node('carcar', 10)

In [52]:
display(root)

0 :
30 foobar
0 barbar:
10 carcar

In [82]:
from types import NoneType


def change_directory(pwd: Union[DirNode, NoneType], argument: str) \
        -> DirNode:
    if argument == '/':
        return DirNode('')
    elif argument == '..':
        assert isinstance(pwd, DirNode)
        display(pwd)
        parent = pwd.contents['..']
        assert isinstance(parent, DirNode)
        # TODO: Sum up the size of pwd
        return parent
    else:  # argument == 'somedir'
        assert isinstance(pwd, DirNode)
        newdir = pwd.contents[argument]
        assert isinstance(newdir, DirNode)
        newdir.contents['..'] = pwd
        return newdir

In [62]:
def parse_ls(ls_output: List[str]) -> Dict[str, Node]:
    output: Dict[str, Node] = dict()
    for line in ls_output:
        stat, name = line.split(' ')
        if stat == 'dir':
            output[name] = DirNode(name)
        else:
            output[name] = Node(name, int(stat))
    return output

In [79]:
def parse_trace(data: List[str]) -> DirNode:
    is_ls = False
    ls_output: List[str] = list()
    file_tree: Union[DirNode, NoneType] = None
    pwd: Union[DirNode, NoneType] = None
    for line in data:
        display(f'{line=}')
        if is_ls and line[0] == '$':
            is_ls = False
            assert isinstance(pwd, DirNode)
            pwd.contents = parse_ls(ls_output)

        if is_ls:
            ls_output.append(line)
        else:
            command, *argument = line[2:].split(' ', 1)
            if command == 'ls':
                is_ls = True
                ls_output = list()
            else:  # command == cd
                pwd = change_directory(pwd, argument[0])
                if not file_tree:
                    file_tree = pwd
    assert isinstance(file_tree, DirNode)
    return file_tree

In [83]:
parse_trace(testdata)

"line='$ cd /'"

"line='$ ls'"

"line='dir a'"

"line='14848514 b.txt'"

"line='8504156 c.dat'"

...

"line='$ ls'"

"line='584 i'"

"line='$ cd ..'"

0 e:
584 i

KeyError: '..'

In [8]:
HTML(downloaded['part1_footer'])

## Part Two

In [9]:
# HTML(downloaded['part2'])

In [10]:
# HTML(downloaded['part2_footer'])